# 导包

In [ ]:
import numpy as np
import pandas as pd
import sqlite3

# 1. 数据组合-concat()函数

In [ ]:
# 1. 加载数据源, 获取df对象.
df1 = pd.read_csv('data/concat_1.csv')
df2 = pd.read_csv('data/concat_2.csv')
df3 = pd.read_csv('data/concat_3.csv')
df1
df2
df3

In [ ]:
# 2. concat()函数, 把df对象连接起来, 默认是: 按行拼接.
# 格式: pd.concat([df1, df2, df3...])
# 细节: concat()函数, 按行拼接是, 参考: 列名,  按列拼接是, 参考: 行索引
pd.concat([df1, df2, df3])  # 按行拼接
pd.concat([df1, df2, df3], axis='rows')  # 按行拼接, 效果同上
pd.concat([df1, df2, df3], axis=0)  # 按行拼接, 效果同上

pd.concat([df1, df2, df3], axis='columns')  # 按列拼接
pd.concat([df1, df2, df3], axis=1)  # 按列拼接, 效果同上

In [ ]:
# 3. 把DataFrame 和 Series对象拼接到一起.
# 细节: 由于Series是列数据, concat()方法默认是添加行, 但是Series中没有行索引, 所以添加了新的列, 缺失值用NaN条虫. 
s1 = pd.Series(['n1', 'n2', 'n3'])
pd.concat([df1, s1])

In [ ]:
# 4. 如果想将 ['n1', 'n2', 'n3', 'n4'] 作为 行链接到 df1后, 如何实现.
df5 = pd.DataFrame([['n1', 'n2', 'n3', 'n4']], columns=['A', 'B', 'C', 'D'])  # 按行拼接, 参考: 列名,  按列拼接, 参考: 行索引
df5

pd.concat([df1, df5], ignore_index=True)  # 忽略行索引, 即: 会自动重置. 

In [ ]:
# 5. 演示: append()函数, 注意: 该函数已过时, 新版本中已移除它. 
df1.append(df2, ignore_index=True)

# 使用python字段, 添加数据行
dict1 = {'A': 'n1', 'B': 'n2', 'C': 'n3', 'D': 'n4'}
dict1

df1.append(dict1, ignore_index=True)  # 把字典元素 拼接到 df1对象中. 
# df1.append(dict1)                     # 报错, 和字典拼接的时候, 必须写 ignore_index=True

In [ ]:
# 6. 添加列. 
pd.concat([df1, df2], axis='columns')
pd.concat([df1, df2], axis=1)           # 效果同上.
# pd.concat([df1, df2], axis=1, ignore_index=True)           # 效果同上.

# df[列名] = 列值  这个方式也可以给df新增1列
df1['new_col'] = 'ai20'
df1

df1['new_col2'] = ['张三', '李四', '王五', '赵六']
df1

# 还可以吧要添加的列值, 封装成: Series对象, 因为: Series对象就是一维数组, 代表着: 某行, 或者某列数据
df1['new_col3'] = pd.Series(['刘亦菲', '胡歌', '高圆圆', '光良'])
df1

# 2. 数据组合-merge()函数

## 2.1 一对一合并

In [ ]:
# 1. 从sqlite中, 读取数据源. 
conn = sqlite3.connect('data/chinook.db')
tracks = pd.read_sql_query('select * from tracks;', conn)    # 歌曲表
tracks.head()

In [ ]:
genres = pd.read_sql_query('select * from genres;', conn)   # 歌曲分类表
genres.head()

In [ ]:
# 2. 为了更好的演示 连接查询, 防止不相关的列来干扰我们, 我们从 tracks(歌曲表中)抽取一些数据.
# 从track表(歌曲表)提取部分数据, 使其不含重复的'GenreID'值
tracks_subset = tracks.loc[[0, 62, 76, 98, 110, 193, 204, 281, 322, 359], ]
tracks_subset

In [ ]:
# 3. 通过 merge()函数, 实现 tracks_subset(歌曲表子集) 和 genres(歌曲分类表) 连接操作.
# 格式: df.merge(df, on='关联字段', how='连接方式')
# 细节1: 如果两个df对象关联字段一样, 用on直接连接.   如果不一样, 用 left_on='左df字段名', right_on='右df字段名'
genres.merge(tracks_subset[['TrackId', 'GenreId', 'Milliseconds']], on='GenreId', how='left')       # left = 左外连接, 左表全集 + 交集.  
genres.merge(tracks_subset[['TrackId', 'GenreId', 'Milliseconds']], on='GenreId', how='right')      # right = 右外连接, 右表全集 + 交集.  
genres.merge(tracks_subset[['TrackId', 'GenreId', 'Milliseconds']], on='GenreId', how='inner')      # inner = 内连接, 交集.  
genres.merge(tracks_subset[['TrackId', 'GenreId', 'Milliseconds']], on='GenreId', how='outer')      # outer = 满外连接(左外连接 + 右外连接), 交集.

In [ ]:
# 细节2: 如果两个df的字段重名了, 则 suffixes=('_x', '_y')会分别给 左df 和 右df加后缀, 以示区分.
# genres.merge(tracks_subset[['TrackId', 'Name', 'GenreId', 'Milliseconds']], on='GenreId', how='outer', suffixes=('_左表', '_右表')) 
genres.merge(tracks_subset[['TrackId', 'Name', 'GenreId', 'Milliseconds']], on='GenreId', how='outer') 

## 2.2 多对一合并

In [90]:
# 需求: 计算 每个类别 歌曲的平均时长.
# 1. 把 歌曲表 tracks 和 歌曲类别表 genres关联到一起.
# genres.merge(tracks_subset)         # 一对一, 因为 tracks_subset(歌曲表子集)中只有10条数据, 且 歌曲分类id都是不同的. 
# genres.merge(tracks)                # 一对多, 因为 tracks(歌曲表)中 多首歌曲  有可能属于 同一个类别.  
genre_track = genres.merge(tracks[['TrackId', 'GenreId', 'Milliseconds']], on='GenreId', how='left')

# 2. 基于上述的数据, 按照 歌曲类别分组, 计算 平均时长.
tmp_series = genre_track.groupby(['GenreId', 'Name'])['Milliseconds'].mean()
tmp_series

# 3. 基于上述的数据, 转成 日期格式.
# pd.to_timedelta(对象, 单位)       把指定的内容(对象) 转成 timedelta 日期类型
# dt.floor()                       这里是做截断的意思, s表示秒, 即: 截断到秒级.
pd.to_timedelta(tmp_series, unit='ms').dt.floor('s').sort_values()

GenreId  Name              
5        Rock And Roll        0 days 00:02:14
25       Opera                0 days 00:02:54
17       Hip Hop/Rap          0 days 00:02:58
12       Easy Listening       0 days 00:03:09
11       Bossa Nova           0 days 00:03:39
14       R&B/Soul             0 days 00:03:40
16       World                0 days 00:03:44
9        Pop                  0 days 00:03:49
7        Latin                0 days 00:03:52
4        Alternative & Punk   0 days 00:03:54
10       Soundtrack           0 days 00:04:04
8        Reggae               0 days 00:04:07
23       Alternative          0 days 00:04:24
6        Blues                0 days 00:04:30
1        Rock                 0 days 00:04:43
2        Jazz                 0 days 00:04:51
24       Classical            0 days 00:04:53
13       Heavy Metal          0 days 00:04:57
15       Electronica/Dance    0 days 00:05:02
3        Metal                0 days 00:05:09
22       Comedy               0 days 00:26:25
19    

# 3. 数据组合-join函数 (了解)

In [92]:
# 1. 读取数据源, 获取df对象.
stocks_2016 = pd.read_csv('data/stocks_2016.csv')
stocks_2016

,Symbol,Shares,Low,High
0,AAPL,80,95,110
1,TSLA,50,80,130
2,WMT,40,55,70


In [94]:
stocks_2017 = pd.read_csv('data/stocks_2017.csv')
stocks_2017

,Symbol,Shares,Low,High
0,AAPL,50,120,140
1,GE,100,30,40
2,IBM,87,75,95
3,SLB,20,55,85
4,TXN,500,15,23
5,TSLA,100,100,300


In [95]:
stocks_2018 = pd.read_csv('data/stocks_2018.csv')
stocks_2018

,Symbol,Shares,Low,High
0,AAPL,40,135,170
1,AMZN,8,900,1125
2,TSLA,50,220,400


In [105]:
# 2. 通过join, 来合并 上述的 df对象.      join: 默认是根据行索引来匹配的, 可以通过 on 设置关联字段. 
# 场景1: 依据两个df的行索引来合并.
stocks_2016.join(stocks_2017, lsuffix='_2016', rsuffix='_2017')                 # 默认是: 左外连接
stocks_2016.join(stocks_2017, lsuffix='_2016', rsuffix='_2017', how='outer')    # 指定: 满外连接

# 场景2: 两个df的symbol设置为行索引, 然后关联. 无需手动写on字段, 默认就是: 按照行索引关联.
stocks_2016.set_index('Symbol').join(stocks_2017.set_index('Symbol'), lsuffix='_2016', rsuffix='_2017')

# 场景3: 1个df的Symbol设置为行索引, 另1个df不设置.
stocks_2016.join(stocks_2018.set_index('Symbol'), lsuffix='_2016', rsuffix='_2017', on='Symbol')

# 上边的事儿, 用: concat(), merge()都能实现.
pd.concat([stocks_2016, stocks_2018], axis='columns')
pd.concat([stocks_2016, stocks_2018], axis=1)           # 行拼接: 参考列名.  列拼接: 参考行索引, 默认为: outer(满外连接)

stocks_2016.merge(stocks_2018, on='Symbol')             # 只能列拼接, 默认为: inner(内连接)

,Symbol,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
0,AAPL,80,95,110,40,135,170
1,TSLA,50,80,130,50,220,400
